<a href="https://colab.research.google.com/github/zeeeeeeeeun/bigdata-dacon/blob/main/motion_cla_bytime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score,precision_score,recall_score, roc_curve, classification_report,precision_recall_curve
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler, Binarizer, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold

import warnings
warnings.filterwarnings(action="ignore")

In [2]:
from google.colab import drive
mp = '/gdrive'
drive.mount(mp, force_remount=True)
path = mp + '/My Drive/Colab Notebooks/motion_cla/'

Mounted at /gdrive


In [3]:
train_features = pd.read_csv(path + "train_features.csv")
train_labels = pd.read_csv(path + "train_labels.csv")
test_features = pd.read_csv(path + "test_features.csv")
submission = pd.read_csv(path + "sample_submission.csv")

In [4]:
print(train_features.shape)
print(train_labels.shape)
print(test_features.shape)
print(submission.shape)

(1875000, 8)
(3125, 3)
(469200, 8)
(782, 62)


In [5]:
train_features = train_features[train_features['time'].isin([0, 100, 200, 300, 400, 500, 600])]
test_features = test_features[test_features['time'].isin([0, 100, 200, 300, 400, 500, 600])]

In [6]:
print(train_features.shape)
print(train_labels.shape)
print(test_features.shape)
print(submission.shape)

(18750, 8)
(3125, 3)
(4692, 8)
(782, 62)


In [7]:
test_features

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,3125,0,-0.628100,-0.160155,0.151487,49.665357,88.435961,13.597668
100,3125,100,-1.553670,0.037591,-0.324577,0.389402,-44.949417,-9.888775
200,3125,200,-1.051346,0.065257,-0.387636,8.661575,-0.299820,3.213252
300,3125,300,-1.313725,0.022755,-0.174541,20.303674,19.560614,-11.266432
400,3125,400,-0.775950,-0.062063,-0.015923,-16.206865,-47.766724,-30.137829
...,...,...,...,...,...,...,...,...
468700,3906,100,-0.765558,-0.319072,-0.241683,-80.883931,-38.258202,-21.549249
468800,3906,200,-0.838335,-0.267716,0.106289,36.935912,9.803008,11.388686
468900,3906,300,-0.866419,-0.419639,0.163305,-19.682486,1.516441,0.266919
469000,3906,400,-0.786409,-0.405281,-0.236529,-86.930713,-91.384269,-57.921560


In [8]:
# 내 구석기 코드 ..
new_train = pd.DataFrame()
temp = pd.DataFrame()

for id in range(0,3125):
  train_id = train_features[train_features['id'] == id]
  # features = ['id', 'acc_x, 1', 'acc_x, 2', 'acc_x, 3', 'acc_x, 4', 'acc_x, 5', 'acc_x, 6', 'acc_y, 1', 'acc_y, 2', 'acc_y, 3', 'acc_y, 4', 'acc_y, 5', 'acc_y, 6', 'acc_z, 1', 'acc_z, 2', 'acc_z, 3', 'acc_z, 4', 'acc_z, 5', 'acc_z, 6', 'gy_x, 1', 'gy_x, 2', 'gy_x, 3', 'gy_x, 4', 'gy_x, 5', 'gy_x, 6', 'gy_y, 1', 'gy_y, 2', 'gy_y, 3', 'gy_y, 4', 'gy_y, 5', 'gy_y, 6', 'gy_z, 1', 'gy_z, 2', 'gy_z, 3', 'gy_z, 4', 'gy_z, 5', 'gy_z, 6']
  # temp[features] = train_id['acc_x'][train_features['time']==0]
  temp['id'] = [id]
  temp = temp.set_index('id')

  temp['acc_x, 1'] = [train_id['acc_x'][train_features['time']==0].values[0]]
  temp['acc_x, 2'] = [train_id['acc_x'][train_features['time']==100].values[0]]
  temp['acc_x, 3'] = [train_id['acc_x'][train_features['time']==200].values[0]]
  temp['acc_x, 4'] = [train_id['acc_x'][train_features['time']==300].values[0]]
  temp['acc_x, 5'] = [train_id['acc_x'][train_features['time']==400].values[0]]
  temp['acc_x, 6'] = [train_id['acc_x'][train_features['time']==500].values[0]]
  
  temp['acc_y, 1'] = [train_id['acc_y'][train_features['time']==0].values[0]]
  temp['acc_y, 2'] = [train_id['acc_y'][train_features['time']==100].values[0]]
  temp['acc_y, 3'] = [train_id['acc_y'][train_features['time']==200].values[0]]
  temp['acc_y, 4'] = [train_id['acc_y'][train_features['time']==300].values[0]]
  temp['acc_y, 5'] = [train_id['acc_y'][train_features['time']==400].values[0]]
  temp['acc_y, 6'] = [train_id['acc_y'][train_features['time']==500].values[0]]

  temp['acc_z, 1'] = [train_id['acc_z'][train_features['time']==0].values[0]]
  temp['acc_z, 2'] = [train_id['acc_z'][train_features['time']==100].values[0]]
  temp['acc_z, 3'] = [train_id['acc_z'][train_features['time']==200].values[0]]
  temp['acc_z, 4'] = [train_id['acc_z'][train_features['time']==300].values[0]]
  temp['acc_z, 5'] = [train_id['acc_z'][train_features['time']==400].values[0]]
  temp['acc_z, 6'] = [train_id['acc_z'][train_features['time']==500].values[0]]

  temp['gy_x, 1'] = [train_id['gy_x'][train_features['time']==0].values[0]]
  temp['gy_x, 2'] = [train_id['gy_x'][train_features['time']==100].values[0]]
  temp['gy_x, 3'] = [train_id['gy_x'][train_features['time']==200].values[0]]
  temp['gy_x, 4'] = [train_id['gy_x'][train_features['time']==300].values[0]]
  temp['gy_x, 5'] = [train_id['gy_x'][train_features['time']==400].values[0]]
  temp['gy_x, 6'] = [train_id['gy_x'][train_features['time']==500].values[0]]

  temp['gy_y, 1'] = [train_id['gy_y'][train_features['time']==0].values[0]]
  temp['gy_y, 2'] = [train_id['gy_y'][train_features['time']==100].values[0]]
  temp['gy_y, 3'] = [train_id['gy_y'][train_features['time']==200].values[0]]
  temp['gy_y, 4'] = [train_id['gy_y'][train_features['time']==300].values[0]]
  temp['gy_y, 5'] = [train_id['gy_y'][train_features['time']==400].values[0]]
  temp['gy_y, 6'] = [train_id['gy_y'][train_features['time']==500].values[0]]

  temp['gy_z, 1'] = [train_id['gy_z'][train_features['time']==0].values[0]]
  temp['gy_z, 2'] = [train_id['gy_z'][train_features['time']==100].values[0]]
  temp['gy_z, 3'] = [train_id['gy_z'][train_features['time']==200].values[0]]
  temp['gy_z, 4'] = [train_id['gy_z'][train_features['time']==300].values[0]]
  temp['gy_z, 5'] = [train_id['gy_z'][train_features['time']==400].values[0]]
  temp['gy_z, 6'] = [train_id['gy_z'][train_features['time']==500].values[0]]

  new_train = new_train.append(temp)

In [9]:
new_train

,"acc_x, 1","acc_x, 2","acc_x, 3","acc_x, 4","acc_x, 5","acc_x, 6","acc_y, 1","acc_y, 2","acc_y, 3","acc_y, 4","acc_y, 5","acc_y, 6","acc_z, 1","acc_z, 2","acc_z, 3","acc_z, 4","acc_z, 5","acc_z, 6","gy_x, 1","gy_x, 2","gy_x, 3","gy_x, 4","gy_x, 5","gy_x, 6","gy_y, 1","gy_y, 2","gy_y, 3","gy_y, 4","gy_y, 5","gy_y, 6","gy_z, 1","gy_z, 2","gy_z, 3","gy_z, 4","gy_z, 5","gy_z, 6"
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.206087,1.161183,1.158168,0.973210,1.093638,1.023491,-0.179371,-0.389665,-0.586620,-0.428058,-0.344989,-0.273965,-0.148447,-0.272900,-0.221962,-0.259721,-0.323044,-0.410992,-0.591608,13.698439,-1.946347,-5.660378,13.961125,8.650000,-30.549010,-17.724581,-39.146044,-5.879261,4.720894,17.633079,-31.676112,-25.369058,-33.738979,-16.202991,31.474076,27.634345
1,-0.211795,-0.187404,-2.090578,-0.270697,-1.216711,-0.813523,-0.078760,-0.777339,-0.204944,-0.811277,0.071998,-0.097311,0.854627,0.558973,-0.358855,0.555718,-0.152063,-0.878485,18.231943,-48.527230,-325.328531,110.060021,36.431940,-2.263466,10.211164,35.763881,-306.613509,40.893012,-38.978167,59.474563,7.390348,31.650861,-214.577627,-156.272490,-11.291172,-0.152830
2,0.241388,-0.982666,0.751793,0.802949,-0.099850,-0.925986,-0.437876,-0.071786,-0.019705,-0.075825,-0.316622,-0.077640,0.797198,0.355059,0.470182,0.327132,0.895141,0.429029,-14.079567,-1.514395,-12.754327,-7.654296,-34.538412,-16.644628,-189.616276,84.311599,153.102736,-25.677012,-134.459532,69.133559,-22.517136,9.018080,-21.759252,0.298693,-16.349705,7.613585
3,-1.066225,-1.034085,-0.839759,-0.801430,-0.765150,-0.853781,-0.280265,-0.359800,0.164764,-0.044093,0.128661,-0.412691,0.003342,0.118695,0.245004,0.466169,0.446024,0.223663,88.024296,25.168606,-21.441550,-4.669341,-2.565812,-18.763281,-19.983130,-4.241002,4.461719,4.691014,32.867587,10.401407,-85.600536,-31.604447,32.289224,-3.809930,-15.574076,-6.906769
4,0.144852,-0.032012,-0.352712,-1.031281,-0.486516,-0.948048,-1.331613,-0.889933,-0.942939,-0.191518,-0.683318,0.093022,-1.231836,0.365340,0.216576,-0.159559,-0.476367,0.335840,-10.166193,-0.005387,-2.086393,-4.196387,-286.222558,-1.166113,73.608178,-0.038199,2.898542,40.621782,54.735695,-0.080599,12.965565,11.048114,-0.634623,-24.339452,4.736563,-0.226674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3120,0.095092,-0.194432,-0.902315,-0.686178,-0.092048,-0.115178,-0.827868,-1.126654,-0.259424,-0.703777,-0.756336,-0.775192,0.542625,0.059259,-0.231252,0.474625,0.513844,0.543620,-15.022263,-87.901488,-27.708365,12.979993,0.877092,0.634876,-21.834084,23.639907,4.232226,-60.336650,5.184807,8.448069,-17.936508,37.067427,1.254160,-46.256980,-5.992192,-2.371696
3121,-0.588936,-0.492739,-1.116993,-0.666591,-0.936591,-0.990971,-0.131407,-0.012395,-0.231208,-0.313407,-0.274141,-0.204432,-0.077380,-0.229752,0.042586,-0.145833,0.006005,-0.116352,-5.894880,122.195672,-6.285622,67.293869,-18.984592,0.113860,-13.970848,10.364403,1.885930,-21.527187,18.134477,-0.243335,-7.800134,-32.985891,0.361003,-31.272266,15.788652,-3.946851
3122,-1.327998,-1.185544,-0.532998,-0.216892,-0.157864,-1.632342,-0.385354,-0.969624,-0.589132,-0.709450,-0.749723,-0.196951,0.379149,0.239257,0.303373,0.164676,0.263347,-0.088694,-206.062841,368.485022,-96.261554,-24.906561,13.589416,64.320350,-289.407404,-298.238849,-271.908814,104.298438,299.762563,298.914663,199.711188,223.369827,122.504553,-16.105729,-42.841316,-87.008090


In [10]:
# 내 구석기 코드 ..
new_test = pd.DataFrame()
temp = pd.DataFrame()

for id in range(3125,3907):
  train_id = test_features[test_features['id'] == id]
  # features = ['id', 'acc_x, 1', 'acc_x, 2', 'acc_x, 3', 'acc_x, 4', 'acc_x, 5', 'acc_x, 6', 'acc_y, 1', 'acc_y, 2', 'acc_y, 3', 'acc_y, 4', 'acc_y, 5', 'acc_y, 6', 'acc_z, 1', 'acc_z, 2', 'acc_z, 3', 'acc_z, 4', 'acc_z, 5', 'acc_z, 6', 'gy_x, 1', 'gy_x, 2', 'gy_x, 3', 'gy_x, 4', 'gy_x, 5', 'gy_x, 6', 'gy_y, 1', 'gy_y, 2', 'gy_y, 3', 'gy_y, 4', 'gy_y, 5', 'gy_y, 6', 'gy_z, 1', 'gy_z, 2', 'gy_z, 3', 'gy_z, 4', 'gy_z, 5', 'gy_z, 6']
  # temp[features] = train_id['acc_x'][train_features['time']==0]
  temp['id'] = [id]
  temp = temp.set_index('id')

  temp['acc_x, 1'] = [train_id['acc_x'][train_features['time']==0].values[0]]
  temp['acc_x, 2'] = [train_id['acc_x'][train_features['time']==100].values[0]]
  temp['acc_x, 3'] = [train_id['acc_x'][train_features['time']==200].values[0]]
  temp['acc_x, 4'] = [train_id['acc_x'][train_features['time']==300].values[0]]
  temp['acc_x, 5'] = [train_id['acc_x'][train_features['time']==400].values[0]]
  temp['acc_x, 6'] = [train_id['acc_x'][train_features['time']==500].values[0]]
  
  temp['acc_y, 1'] = [train_id['acc_y'][train_features['time']==0].values[0]]
  temp['acc_y, 2'] = [train_id['acc_y'][train_features['time']==100].values[0]]
  temp['acc_y, 3'] = [train_id['acc_y'][train_features['time']==200].values[0]]
  temp['acc_y, 4'] = [train_id['acc_y'][train_features['time']==300].values[0]]
  temp['acc_y, 5'] = [train_id['acc_y'][train_features['time']==400].values[0]]
  temp['acc_y, 6'] = [train_id['acc_y'][train_features['time']==500].values[0]]

  temp['acc_z, 1'] = [train_id['acc_z'][train_features['time']==0].values[0]]
  temp['acc_z, 2'] = [train_id['acc_z'][train_features['time']==100].values[0]]
  temp['acc_z, 3'] = [train_id['acc_z'][train_features['time']==200].values[0]]
  temp['acc_z, 4'] = [train_id['acc_z'][train_features['time']==300].values[0]]
  temp['acc_z, 5'] = [train_id['acc_z'][train_features['time']==400].values[0]]
  temp['acc_z, 6'] = [train_id['acc_z'][train_features['time']==500].values[0]]

  temp['gy_x, 1'] = [train_id['gy_x'][train_features['time']==0].values[0]]
  temp['gy_x, 2'] = [train_id['gy_x'][train_features['time']==100].values[0]]
  temp['gy_x, 3'] = [train_id['gy_x'][train_features['time']==200].values[0]]
  temp['gy_x, 4'] = [train_id['gy_x'][train_features['time']==300].values[0]]
  temp['gy_x, 5'] = [train_id['gy_x'][train_features['time']==400].values[0]]
  temp['gy_x, 6'] = [train_id['gy_x'][train_features['time']==500].values[0]]

  temp['gy_y, 1'] = [train_id['gy_y'][train_features['time']==0].values[0]]
  temp['gy_y, 2'] = [train_id['gy_y'][train_features['time']==100].values[0]]
  temp['gy_y, 3'] = [train_id['gy_y'][train_features['time']==200].values[0]]
  temp['gy_y, 4'] = [train_id['gy_y'][train_features['time']==300].values[0]]
  temp['gy_y, 5'] = [train_id['gy_y'][train_features['time']==400].values[0]]
  temp['gy_y, 6'] = [train_id['gy_y'][train_features['time']==500].values[0]]

  temp['gy_z, 1'] = [train_id['gy_z'][train_features['time']==0].values[0]]
  temp['gy_z, 2'] = [train_id['gy_z'][train_features['time']==100].values[0]]
  temp['gy_z, 3'] = [train_id['gy_z'][train_features['time']==200].values[0]]
  temp['gy_z, 4'] = [train_id['gy_z'][train_features['time']==300].values[0]]
  temp['gy_z, 5'] = [train_id['gy_z'][train_features['time']==400].values[0]]
  temp['gy_z, 6'] = [train_id['gy_z'][train_features['time']==500].values[0]]

  new_test = new_test.append(temp)

In [11]:
new_test

,"acc_x, 1","acc_x, 2","acc_x, 3","acc_x, 4","acc_x, 5","acc_x, 6","acc_y, 1","acc_y, 2","acc_y, 3","acc_y, 4","acc_y, 5","acc_y, 6","acc_z, 1","acc_z, 2","acc_z, 3","acc_z, 4","acc_z, 5","acc_z, 6","gy_x, 1","gy_x, 2","gy_x, 3","gy_x, 4","gy_x, 5","gy_x, 6","gy_y, 1","gy_y, 2","gy_y, 3","gy_y, 4","gy_y, 5","gy_y, 6","gy_z, 1","gy_z, 2","gy_z, 3","gy_z, 4","gy_z, 5","gy_z, 6"
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3125,-0.628100,-1.553670,-1.051346,-1.313725,-0.775950,-0.935534,-0.160155,0.037591,0.065257,0.022755,-0.062063,-0.064341,0.151487,-0.324577,-0.387636,-0.174541,-0.015923,-0.378459,49.665357,0.389402,8.661575,20.303674,-16.206865,-6.123668,88.435961,-44.949417,-0.299820,19.560614,-47.766724,-6.460440,13.597668,-9.888775,3.213252,-11.266432,-30.137829,14.436183
3126,0.304222,0.132422,-0.970376,-0.243782,0.598151,-0.746997,1.529324,1.151095,0.419156,0.876904,0.283841,0.574812,-0.338265,-0.351002,-0.570795,0.170640,0.625881,0.088497,62.204081,-37.363267,11.181830,-47.367416,-18.045566,-41.936050,-20.624492,103.239094,-10.836717,-1.996642,-14.650808,23.619766,-65.532643,-94.602060,-29.127357,48.140277,8.106511,5.563665
3127,0.637535,0.557659,0.470364,0.505726,0.427501,0.494818,0.727123,0.954322,0.630532,1.022548,1.043605,0.771408,-0.151076,-0.197033,-0.031752,-0.193501,-0.345815,-0.104170,-49.099601,29.945749,10.902969,-27.864387,0.165850,-99.799971,-38.428781,16.189134,-21.725602,13.238919,-9.357536,-24.259958,-4.451638,81.935362,-16.995632,13.577619,-3.513435,28.793659
3128,-1.022308,-0.803196,-0.580076,-0.092435,-0.863330,-0.865098,-0.373770,-0.593992,-0.528921,-1.002368,-0.481961,-0.509056,-0.227997,0.358140,0.524574,-0.435935,-0.089883,-0.088354,-7.017024,-5.530923,3.008136,4.105380,8.263500,3.375282,27.106675,-143.090723,6.740738,4.212964,7.298540,4.584097,6.714589,-65.413983,-39.972575,-0.157441,-0.432398,1.518809
3129,-0.270255,-1.201815,-0.793753,-0.716452,-0.663341,-0.581294,0.172035,0.089424,-0.105279,0.062178,0.348797,1.087729,0.677061,-0.108263,0.518678,0.606076,0.918095,0.429411,-166.081005,-11.341067,-60.591626,4.628335,103.586432,-149.031531,35.340279,-64.220529,-57.152997,73.087901,80.085223,-15.102027,137.892264,22.404907,42.111646,9.066969,6.415480,-58.916674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3902,-0.355658,-0.888741,-0.953686,-1.077982,-1.233253,-0.785488,1.160691,0.419192,0.450914,0.329238,0.337364,-0.004023,-0.121460,-0.332601,-0.109695,-0.207794,-0.199901,-0.296355,147.862998,-22.965237,93.297026,56.460410,254.306142,347.011233,131.915912,-57.340881,-25.259935,-28.582111,-148.763906,-52.052673,-108.257915,-31.331823,-9.486073,-33.536423,-119.141954,36.344515
3903,-0.292415,-1.356575,-0.701057,-1.051379,-1.126982,-0.450873,0.764595,0.483186,0.609517,0.064407,0.351247,0.885593,-0.105775,-0.066077,-0.328074,0.126267,-0.281558,-0.235992,38.355670,9.813654,-72.035051,-94.602554,22.903997,-43.952115,-40.841240,-21.281799,25.006661,43.281191,-4.259780,15.962131,92.087589,12.530988,114.103698,34.086139,79.801914,-26.826459
3904,-1.039463,-0.963679,-0.970639,-0.787790,-0.791070,-0.447065,-0.061784,0.004932,-0.140531,0.118908,0.180791,0.624662,-0.029856,0.041197,0.191326,0.462096,0.410772,0.531423,44.132684,28.151443,-6.282463,-33.138224,10.330638,-0.950346,-129.695853,20.665661,-76.979567,12.336672,-7.030115,-5.415836,85.555965,-35.431527,51.194663,2.771600,1.920407,10.551022


In [12]:
X_train = new_train
X_test = new_test

In [13]:
y_train = train_labels['label']
y_train

0       37
1       26
2        3
3       26
4       26
        ..
3120    26
3121    26
3122    15
3123    26
3124     2
Name: label, Length: 3125, dtype: int64

In [14]:
model = RandomForestClassifier(n_jobs=-1, random_state=121, min_samples_leaf=1, max_depth=10, n_estimators=400)

In [15]:
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=-1, oob_score=False, random_state=121, verbose=0,
                       warm_start=False)

In [16]:
y_pred = model.predict_proba(X_test)
y_pred

array([[3.74522761e-03, 1.82750632e-04, 1.33950866e-03, ...,
        9.35900459e-03, 5.08729898e-05, 2.49928877e-02],
       [1.02236555e-02, 5.97500896e-03, 2.97252479e-03, ...,
        4.83881142e-03, 2.19696573e-03, 1.04000618e-02],
       [6.48264871e-03, 6.98466211e-02, 3.50786275e-03, ...,
        0.00000000e+00, 4.29712665e-02, 4.60988499e-03],
       ...,
       [2.64565775e-03, 2.49981679e-04, 4.88252106e-03, ...,
        6.11475330e-03, 4.14893415e-04, 1.90834321e-02],
       [2.85992264e-03, 3.89917786e-02, 2.59589606e-03, ...,
        0.00000000e+00, 4.62839598e-02, 1.70299581e-03],
       [9.80906785e-04, 3.32036036e-04, 4.65935387e-03, ...,
        1.11458969e-02, 4.72761894e-04, 1.66412676e-02]])

In [17]:
submission.iloc[:,1:] = y_pred
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0.003745,0.000183,0.001340,0.002086,0.012000,0.002543,0.000000,0.001309,0.005593,0.012868,0.011521,0.008299,0.002663,0.021726,0.011841,0.005921,0.000946,0.000071,0.005470,0.006354,0.014881,0.004784,0.003191,0.007236,0.017830,0.004154,0.683775,0.000160,0.000961,0.000824,0.012405,0.004107,0.006470,0.001708,0.013868,0.000558,0.011263,0.000000,0.001753,0.000606,0.002888,0.000657,0.002470,0.001615,0.000065,0.000078,0.005055,0.000036,0.006222,0.005344,0.006037,0.004412,0.003170,0.005101,0.000247,0.002229,0.000659,0.012301,0.009359,0.000051,0.024993
1,3126,0.010224,0.005975,0.002973,0.008534,0.006104,0.026225,0.000559,0.015232,0.009386,0.005185,0.003809,0.001831,0.000569,0.001111,0.001731,0.004184,0.022537,0.003963,0.018546,0.000562,0.008077,0.002478,0.004641,0.004031,0.002596,0.004748,0.555322,0.014272,0.007048,0.002543,0.002913,0.001468,0.004955,0.008917,0.003824,0.009353,0.006741,0.007658,0.006508,0.000436,0.024699,0.016066,0.003274,0.008530,0.008480,0.008982,0.000920,0.000518,0.003333,0.006082,0.017955,0.001757,0.000477,0.001666,0.014679,0.013394,0.041462,0.002518,0.004839,0.002197,0.010400
2,3127,0.006483,0.069847,0.003508,0.003297,0.000605,0.006175,0.006887,0.028853,0.115152,0.000150,0.000000,0.000261,0.000000,0.000318,0.000000,0.002215,0.020479,0.013892,0.017983,0.000833,0.011178,0.002522,0.000038,0.002920,0.000041,0.012280,0.127191,0.048958,0.009413,0.018704,0.000055,0.000014,0.009086,0.004681,0.002444,0.025758,0.005960,0.034897,0.000908,0.001074,0.025733,0.008567,0.010885,0.018089,0.034271,0.074595,0.002361,0.013067,0.014982,0.014645,0.002082,0.001681,0.000013,0.000066,0.025771,0.023201,0.066990,0.000360,0.000000,0.042971,0.004610
3,3128,0.002805,0.000224,0.008520,0.004972,0.008887,0.004079,0.000000,0.002316,0.022715,0.010283,0.008073,0.008420,0.003904,0.003444,0.010303,0.005207,0.004444,0.000096,0.002673,0.001406,0.004735,0.014497,0.005027,0.004530,0.020453,0.001158,0.647230,0.000315,0.003980,0.003317,0.012122,0.007195,0.006007,0.004612,0.010296,0.002438,0.008305,0.000115,0.002532,0.001320,0.000979,0.005011,0.007649,0.006971,0.000222,0.000359,0.001878,0.000035,0.008697,0.012398,0.016875,0.007140,0.005307,0.003519,0.000565,0.013267,0.001209,0.006708,0.008986,0.000321,0.018951
4,3129,0.007040,0.002811,0.001811,0.006751,0.010255,0.014102,0.000000,0.001728,0.009589,0.006992,0.003321,0.006612,0.000414,0.002557,0.002933,0.002845,0.005383,0.000466,0.007574,0.002682,0.001932,0.007165,0.010878,0.002544,0.003854,0.003694,0.738276,0.000508,0.001329,0.000894,0.008449,0.003990,0.002718,0.002200,0.006948,0.004248,0.005636,0.000000,0.000673,0.000087,0.001493,0.000954,0.003193,0.005490,0.000052,0.003031,0.000946,0.000071,0.005157,0.003988,0.023963,0.005542,0.000304,0.008578,0.000819,0.007277,0.004415,0.013175,0.001907,0.000470,0.007286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0.003305,0.000018,0.003827,0.005231,0.016776,0.004282,0.000000,0.001687,0.003451,0.011192,0.005225,0.006911,0.002712,0.004184,0.003592,0.004710,0.003493,0.002563,0.027418,0.012404,0.004215,0.010141,0.005522,0.005866,0.010378,0.010418,0.667259,0.003255,0.000407,0.000716,0.008088,0.003835,0.005535,0.001773,0.015035,0.000657,0.019234,0.000015,0.001246,0.000581,0.006070,0.009685,0.002080,0.001978,0.000068,0.000147,0.005695,0.000019,0.004658,0.003731,0.007294,0.007378,0.002504,0.011634,0.000261,0.002960,0.006610,0.008022,0.004528,0.000243,0.017278
778,3903,0.003529,0.000174,0.002343,0.004760,0.008278,0.003891,0.000000,0.001965,0.003891,0.011366,0.003655,0.010812,0.001682,0.007507,0.005004,0.003247,0.001962,0.000933,0.005942,0.003243,0.013436,0.007450,0.005084,0.006039,0.010090

In [18]:
submission.to_csv('rf_time_e400_md10_msl1_rs121.csv', index=False)

In [19]:
my_hyper_param = {  "n_estimators"     :[300, 400] ,
                    "max_depth"        :[10],
                    "min_samples_leaf" :[1],
                    "random_state"     :[121]
                 }

In [20]:
from sklearn.metrics import log_loss, make_scorer

LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)
gcv_model = GridSearchCV(model, param_grid=my_hyper_param, scoring=LogLoss, refit=True, cv=5, verbose=0)

In [21]:
gcv_model.fit(X_train, y_train)
print("best_estimator_", gcv_model.best_estimator_)
print("best_params_",    gcv_model.best_params_)
print("best_score_" ,    gcv_model.best_score_)
y_pred = gcv_model.predict_proba(X_test).astype(np.float64)
print(y_pred)
# best_params_ {'max_depth': 10, 'min_samples_leaf': 1, 'n_estimators': 400, 'n_jobs': -1, 'random_state': 121}
# best_score_ -1.829393965808745

best_estimator_ RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=-1, oob_score=False, random_state=121, verbose=0,
                       warm_start=False)
best_params_ {'max_depth': 10, 'min_samples_leaf': 1, 'n_estimators': 400, 'random_state': 121}
best_score_ -1.829393965808745
[[3.74522761e-03 1.82750632e-04 1.33950866e-03 ... 9.35900459e-03
  5.08729898e-05 2.49928877e-02]
 [1.02236555e-02 5.97500896e-03 2.97252479e-03 ... 4.83881142e-03
  2.19696573e-03 1.04000618e-02]
 [6.48264871e-03 6.98466211e-02 3.50786275e-03 ... 0.00000000e+00
  4.29712665e-02 4.60988499e-03]
 ...
 [2.64565775e